In [1]:
import pandas as pd
import bs4 as bs
import urllib.request
from datetime import datetime

from finance_data_extract import get_long_historical_price

C:\Users\nilay\Anaconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [104]:
df=get_long_historical_price("^GSPC", datetime.strptime('Jan 1 1990', '%b %d %Y').date()
                     , datetime.now().date()
                     , frequency='1d',display=True)


1990-01-01 631087200  + 86,400 =  631173600
1990-03-31 638859600


C:\Users\nilay\Desktop\project-2-Term-Structure\Data\finance_data_extract.py:78: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  extract_table_df = extract_table_df.convert_objects(convert_numeric=True)



1990-04-01 638859600  + 86,400 =  638946000
1990-06-30 646722000

1990-07-01 646722000  + 86,400 =  646808400
1990-09-30 654670800

1990-10-01 654670800  + 86,400 =  654757200
1990-12-31 662623200

1991-01-01 662623200  + 86,400 =  662709600
1991-03-31 670395600

1991-04-01 670395600  + 86,400 =  670482000
1991-06-30 678258000

1991-07-01 678258000  + 86,400 =  678344400
1991-09-30 686206800

1991-10-01 686206800  + 86,400 =  686293200
1991-12-31 694159200

1992-01-01 694159200  + 86,400 =  694245600
1992-03-31 702018000

1992-04-01 702018000  + 86,400 =  702104400
1992-06-30 709880400

1992-07-01 709880400  + 86,400 =  709966800
1992-09-30 717829200

1992-10-01 717829200  + 86,400 =  717915600
1992-12-31 725781600

1993-01-01 725781600  + 86,400 =  725868000
1993-03-31 733554000

1993-04-01 733554000  + 86,400 =  733640400
1993-06-30 741416400

1993-07-01 741416400  + 86,400 =  741502800
1993-09-30 749365200

1993-10-01 749365200  + 86,400 =  749451600
1993-12-31 757317600

1994-01-0

In [105]:
df.to_csv('sp500.csv')

In [9]:
df = pd.read_csv('sp500.csv', sep=",",thousands=',')

In [10]:
df['Date']=pd.to_datetime(df['Date'])

In [11]:
df = df.set_index('Date')

In [15]:
df['6 month return']=df['Adj Close'].pct_change(periods=126)
df['1 year return']=df['Adj Close'].pct_change(periods=251)
df['3 year return']=df['Adj Close'].pct_change(periods=753)
df['5 year return'] = df['Adj Close'].pct_change(periods=1255)
df['10 year return']=df['Adj Close'].pct_change(periods=1510)
df.head()

,Open,High,Low,Close,Adj Close,Volume,1 year return,3 year return,5 year return,10 year return,6 month return
Date,,,,,,,,,,,
1990-01-02,353.40,359.69,351.98,359.69,359.69,162070000,NaN,NaN,NaN,NaN,NaN
1990-01-03,359.69,360.59,357.89,358.76,358.76,192330000,NaN,NaN,NaN,NaN,NaN
1990-01-04,358.76,358.76,352.89,355.67,355.67,177000000,NaN,NaN,NaN,NaN,NaN
1990-01-05,355.67,355.67,351.35,352.20,352.20,158530000,NaN,NaN,NaN,NaN,NaN
1990-01-08,352.20,354.24,350.54,353.79,353.79,140110000,NaN,NaN,NaN,NaN,NaN


In [16]:
database_df = df[['6 month return', '1 year return', '3 year return', '5 year return', '10 year return']]

In [18]:
database_df

,6 month return,1 year return,3 year return,5 year return,10 year return
id,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN
1990-01-09,NaN,NaN,NaN,NaN,NaN
1990-01-10,NaN,NaN,NaN,NaN,NaN
1990-01-11,NaN,NaN,NaN,NaN,NaN
1990-01-12,NaN,NaN,NaN,NaN,NaN


In [19]:
conn = sqlite3.connect('sp500_return.db')
database_df.to_sql(name='rolling_returns', con=conn, if_exists='replace')
conn.close()

NameError: name 'sqlite3' is not defined